https://sagemaker-examples.readthedocs.io/en/latest/frameworks/tensorflow/get_started_mnist_train.html#TensorFlow-Estimator

In [1]:
import os
import json
from datetime import datetime

import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
!pygmentize '../trainer/autoencoder.py'

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense
import os, gc
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import numpy as np
import pandas as pd
import argparse, datetime
from tqdm import tqdm

def load_dataset(s3_loc, s3_filename):
    
    data = np.load(os.path.join(s3_loc, s3_filename))
    feature_cols = [f"feature_{i:02d}" for i in range(79)]
    features = {col: data[col] for col in feature_cols}
    x = np.column_stack([features[col] for col in feature_cols])
    print(f"Input shape: {x.shape}")
    del features
    gc.collect()
    
    return x

class AutoEncoderTrainOnly:
    def __init__(self, x_train, params):      
        self.x_train = x_train
        self.input_size = x_train.shape[1]
        self.latent_dim = params["latent_dim"]
        
        self.n_enc_1 = params["n_enc_1"]
        self.n_enc_2 = params["n_enc_2"]
        self.n_dec_1 = params["n_dec_1"]
        self.n_dec_2 = params["n_dec_2"

In [3]:
# Set local_mode to be True if you want to run the training script on the machine that runs this notebook
local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.g4dn.8xlarge"
    
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")[2:17]
output_path = "s3://kedrobucket/supply_chain_data_asset_shr/training/jane_st_forecasting"
# model_dir = os.path.join(output_path, f"tensorflow_training_{timestamp}")
# print(model_dir)
est = TensorFlow(
    entry_point="autoencoder.py",
    source_dir="../trainer",  # directory of your training script
    role=role,
    framework_version="2.16",
    model_dir=False,
    py_version="py310",
    instance_type=instance_type,
    instance_count=1,
    volume_size=80,
    output_path=output_path,
    hyperparameters={
        "batch_size": 256,
        "epochs": 10,
        "learning_rate": 1e-3,
        "s3_filename": "df_530_lite.npz",
    },
)

In [4]:
channels = {"training": output_path}

In [5]:
est.fit(inputs=channels)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-10-25-03-34-40-506


2024-10-25 03:34:43 Starting - Starting the training job...
2024-10-25 03:34:59 Starting - Preparing the instances for training...
2024-10-25 03:35:32 Downloading - Downloading input data.....................
2024-10-25 03:39:04 Training - Training image download completed. Training in progress.../usr/local/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/local/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
2024-10-25 03:39:21.795057: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-

In [8]:
# Download outputs to local
local_path = "../../outputs/"+est.model_data.split("jane_st_forecasting/")[1]
s3_path = est.model_data

!aws s3 cp {s3_path} {local_path}

# Extract downloaded outputs

import tarfile

# Specify the tar file and the directory to extract it to
extract_to_dir = 'model'

# Open the tar file
with tarfile.open(local_path, "r:gz") as tar:  # Use "r" for uncompressed .tar files
    tar.extractall(path=local_path.replace(".tar.gz", ""))

download: s3://kedrobucket/supply_chain_data_asset_shr/training/jane_st_forecasting/tensorflow-training-2024-10-25-03-34-40-506/output/model.tar.gz to ../../outputs/tensorflow-training-2024-10-25-03-34-40-506/output/model.tar.gz
